<p align="justify">
    <picture>
        <source srcset="https://icra25lasat.ieeechile.cl/images/icra25la.png">
        <img alt="nerfstudio" src="	https://icra25lasat.ieeechile.cl/images/icra25la.png" width="400">
    </picture>
    <picture>
        <img alt="nerfstudio" src="https://icra25lasat.ieeechile.cl/images/uoh.png" width="100">
    </picture>
</p>

<p align="left">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/_images/logo.png" width="400">
    </picture>
</p>
# Nerfstudio: A collaboration friendly studio for NeRFs frameworks


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
# @title Install Pixi { vertical-output: true }
from IPython.display import Markdown, display
from rich.console import Console
from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn
import subprocess
import os

# Ensure rich is installed (this step won't be shown in progress)
subprocess.run("pip install rich", shell=True, check=True)

console = Console()
step_cmds = [
    ("Change to /content", ["cd /content/"]),
    ("Download & Install Pixi", ["curl -fsSL https://pixi.sh/install.sh | sh"]),
    ("Add Pixi to PATH", []),  # handled in Python
    ("Verify Pixi version", ["pixi --version"]),
    ("Initialize Pixi Project", ["pixi init ."]),
]

display(Markdown("**Installing Pixi...**"))
with Progress(
    TextColumn(":sparkles: {task.description}"),
    BarColumn(),
    TaskProgressColumn(),
    TimeRemainingColumn(),
    console=console,
    transient=True,
) as progress:
    task = progress.add_task("Total steps", total=len(step_cmds))
    for desc, cmds in step_cmds:
        console.print(f"\n[bold cyan]Step:[/] {desc}")
        if desc == "Add Pixi to PATH":
            # Modify PATH in current session
            os.environ["PATH"] += ":/root/.pixi/bin"
        else:
            for cmd in cmds:
                # use bash for cd commands
                subprocess.run(cmd, shell=True, check=True, cwd="/content/", env=os.environ)
        progress.advance(task)

# Final confirmation
display(Markdown("**Pixi installation complete!**"))


In [ ]:
# @title # Dependencies { vertical-output: true }
# @markdown <h1>Install Nerfstudio and Dependencies (~8 min)</h1>

from IPython.display import Markdown, display
from rich.console import Console
from rich.progress import Progress, TextColumn, BarColumn, TaskProgressColumn, TimeRemainingColumn
import subprocess
import time
import os

# Initialize Rich console and progress
console = Console()
steps = [
    ("Add Python & Pip to Pixi", ["pixi add --manifest-path pixi.toml python=3.10 pip"]),
    ("Upgrade Pip", ["pixi run --manifest-path pixi.toml pip install --upgrade pip"]),
    ("Install PyTorch & Torchaudio", [
        "pixi run --manifest-path pixi.toml pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 \
            torchaudio==2.0.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118"
    ]),
    ("Download TinyCudaNN Wheel", [
        "gdown https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
    ]),
    ("Install TinyCudaNN", [
        "pixi run --manifest-path pixi.toml pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl"
    ]),
    ("Install COLMAP", ["pixi run --manifest-path pixi.toml apt-get install -y colmap"]),
    ("Install Nerfstudio", [
        "pixi run --manifest-path pixi.toml pip install git+https://github.com/nerfstudio-project/nerfstudio.git"
    ]),
    ("Solving Numpy Bug", ["pixi run --manifest-path pixi.toml pip install numpy==1.26.4"]),
]

display(Markdown("**Installing dependencies...**"))

with Progress(
    TextColumn("[:rocket:] {task.description}"),
    BarColumn(),
    TaskProgressColumn(),
    TimeRemainingColumn(),
    console=console,
    transient=True,
) as progress_bar:
    task = progress_bar.add_task("Total steps", total=len(steps))
    for desc, cmds in steps:
        # Display step description
        console.print(f"\n[bold cyan]Step:[/] {desc}")
        for cmd in cmds:
            # Run each shell command
            process = subprocess.run(cmd, shell=True, cwd="/content/", env=os.environ)
            if process.returncode != 0:
                console.print(f"[bold red]Error running:[/] {cmd}")
                raise RuntimeError(f"Step failed: {desc}")
            if "TinyCudaNN Wheel" in desc:
                time.sleep(10)
        progress_bar.advance(task)

# Final message
display(Markdown("**DONE**"))


In [ ]:
# @title # Downloading and Processing Data { vertical-output : true }
# @markdown <h3>Pick the preset scene or upload your own images/video</h3>
import glob
import os

from google.colab import files
from IPython.core.display import HTML, display

scene = "\ud83d\uddbc poster"  # @param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = " ".join(scene.split(" ")[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !pixi run --manifest-path pixi.toml ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == "upload your own Record3D data":
    display(HTML("<h3>Zip your Record3D folder, and upload.</h3>"))
    display(
        HTML(
            '<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'
        )
    )
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob("/content/data/nerfstudio/custom_data/*")[0]
    !pixi run --manifest-path pixi.toml ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ["upload your images", "upload your own video"]:
    display(HTML("<h3>Select your custom data</h3>"))
    display(HTML("<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>"))
    display(
        HTML(
            "<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>"
        )
    )
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == "upload your images":
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == "upload your images":
        !pixi run --manifest-path pixi.toml ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !pixi run --manifest-path pixi.toml ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !pixi run --manifest-path pixi.toml ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

In [ ]:
# @title # Start Training (~ 10 min every 10k train iterations){ vertical-output : true }

from IPython.display import Markdown, display

import os
import subprocess
from collections import deque

from rich.console import Console
from rich.live import Live
from rich.panel import Panel

console = Console()

display(Markdown("[viser generated link](https://xxxxxxxxx.share.viser.studio/)"))

# Example command that streams output
cmd = (
    "pixi run --manifest-path pixi.toml "
    "ns-train nerfacto "
    "--viewer.websocket-port 7007 "
    "--viewer.make-share-url True "
    "--max-num-iterations 10000 "
    f"nerfstudio-data --data data/nerfstudio/{scene} "
    "--downscale-factor 4 "
)

# Prepare a deque to hold up to 10 lines
last_lines = deque(maxlen=15)

# Launch the process
process = subprocess.Popen(
    cmd,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    env={**os.environ, "PYTHONUNBUFFERED": "1"},
)

# Use Live to continuously update the panel
with Live(Panel("Waiting for output..."), console=console, refresh_per_second=4) as live:
    for line in process.stdout:
        # Strip and append the new line
        last_lines.append(line.rstrip())
        # Update the panel with the joined last 10 lines
        live.update(Panel("\n".join(last_lines)))

# Wait for the process to finish
process.wait()
console.print(f"[bold green]Process exited with code {process.returncode}[/]")



In [ ]:
# @title # Render Video { vertical-output: true }
# @markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
# @markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import HTML, display

display(HTML("<h3>Upload the camera path JSON.</h3>"))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!pixi run --manifest-path pixi.toml ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4


# Challenge: Create and train your own datset

## Steps:

* Record a Video
* Use ns-process-data
    * Tip: To analyze "Downloading and Processing Data"
        * !pixi run --manifest-path pixi.toml ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
* Train Nerfacto with your own data
* Optional: Render a Video!

In [ ]:
#@title <h2>Capture and then Upload the Video</h2>

from google.colab import files

# Opens a file‑picker dialog; returns a dict of filename→file‑bytes
%cd /content/
uploaded_video = files.upload()

In [ ]:
# @title # Process Data
# @markdown ## Complete the Code

"""To specify the video path"""
video_path = "/content/..."

!pixi run --manifest-path pixi.toml ns-process-data video --data $video_path  --output-dir """To specify your output path"""


In [ ]:
# @title # Training Nerf
# @markdown ## Complete the Code

data_path = "/content/..."

!pixi run --manifest-path pixi.toml ns-train nerfacto --viewer.websocket-port 7007 --viewer.make-share-url True --max-num-iterations 10000 nerfstudio-data --data $data_path --downscale-factor 4
